# RNN for 3D keypoint generation

## Build network with Keras

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import importlib
import time
import math
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import rc
from matplotlib import rcParams
from sklearn.model_selection import train_test_split
rc('animation', html='html5')
rcParams['animation.embed_limit'] = 2**128
import mdn

# Hacky. Is there a better way to load local modules?

#%run ../renderer/common.py

# still hacky but less hacky
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,os.path.join(parentdir, 'renderer')) 
from common import *

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Import keypoint sequences

In [ ]:
data = []
basedir = '../data/train/'
for video_folder in os.listdir(basedir):
    for folder in os.listdir(basedir + video_folder):
        keypoints = np.load(basedir + video_folder + "/" + folder + "/" + 'pose3d.npz', allow_pickle=True)
        keypoints = keypoints['arr_0']
        data.append(keypoints)


In [ ]:
np.shape(data[0])

### Check data

**Note:** In order to render a video based on the pose3D data, please run `python renderer/render_pose3d_matplot.py ../data/train/seq_001/pose3d.npz --frames 300`

In [ ]:
fig, ax = init_plot()
plot_pose(ax, data[9][100])

In [ ]:
fig, ax = init_plot()
def animate(i):
    ax.clear()
    plot_pose(ax, data[0][i])

anim = FuncAnimation(fig, animate, frames=range(0,len(keypoints), 20), interval=200)
HTML(anim.to_html5_video())

In [ ]:
def create_dataset(data, look_back = 1):
    dataX, dataY = [], []
    for dataset in data:
        # reshape input to be [samples, features = (keypoints*3dim)] 
        dataset = np.reshape(dataset,  (dataset.shape[0], dataset.shape[1]*dataset.shape[2]))
        for i in range(len(dataset) - look_back-1):
            # dataX has dimension [samples, lookback, features = (keypoints*3dim)] 
            a = dataset[i:(i + look_back), :]     
            dataX.append(a)
            # dataY has dimension [samples, features = (keypoints*3dim)] 
            dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)


look_back = 10
X, y = create_dataset(data,look_back)

In [ ]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle= True, random_state=42)

# build model

In [ ]:
model = keras.Sequential()
model.add(layers.LSTM(32, input_shape=(look_back, 51),  return_sequences=True))
model.add(layers.LSTM(32, return_sequences=True))
model.add(layers.LSTM(32))
model.add(layers.Dense(32))
model.add(mdn.MDN(51, 3))
model.compile(loss=mdn.get_mixture_loss_func(51,3), optimizer=keras.optimizers.Adam())
model.summary()


In [ ]:
BATCH_SIZE = 128
limit = int(np.floor(len(X_train)/BATCH_SIZE))*BATCH_SIZE
EPOCHS = 500
VAL_SPLIT = 0.05

In [ ]:
# Train the model

# Define callbacks
filepath="models/lstm_mdn-E{epoch:02d}-VL{val_loss:.2f}.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, save_weights_only=True, verbose=1, save_best_only=True, mode='min')
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
callbacks = [keras.callbacks.TerminateOnNaN(), checkpoint, early_stopping]

history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=callbacks, validation_split=VAL_SPLIT)

# Save the Model
model.save('models/lstm-mdn.h5')  # creates a HDF5 file of the model

# Plot the loss
%matplotlib inline
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [ ]:
y_test = model.predict(x_test)
y_samples = np.apply_along_axis(sample_from_output, 1, y_test, OUTPUT_DIMS, N_MIXES, temp=1.0)

In [ ]:
%matplotlib inline
from IPython.display import HTML

def show_animation(dataset):
    fig, ax = init_plot()
    trainPredict_reshaped = np.reshape(dataset,(np.shape(dataset)[0],17,3))
    def animate(i):
        ax.clear()
        plot_pose(ax, trainPredict_reshaped[i])

    anim = FuncAnimation(fig, animate, frames=range(0,len(dataset),3), interval=200)
    plt.close()
    return HTML(anim.to_jshtml());

In [ ]:
import matplotlib.cm as cm

def plot_pose(ax, frame):
    ax.clear()
    ax.set_xlim3d(-1, 1)
    ax.set_ylim3d(-1, 1)
    ax.set_zlim3d(0, 2)

    for i,kp in enumerate(frame):
        ax.scatter(kp[0], kp[1], kp[2], color=cm.plasma(i/17))

    for (j_from, j_to) in bones:
        ax.plot3D(
            [frame[j_from][0], frame[j_to][0]],
            [frame[j_from][1], frame[j_to][1]],
            [frame[j_from][2], frame[j_to][2]],
            'gray')


In [ ]:
def generate_performance(model, initial_positions, steps_limit=100, n_mixtures=3, temp=1.0, sigma_temp=0.0, look_back=10):
    """Generates aperformance"""
    time = 0
    steps = 0
    performance = [pose for pose in initial_positions]
    while (steps < steps_limit):
        params = model.predict(np.expand_dims(np.array(performance[-10:]), axis=0))
        new_pose = mdn.sample_from_output(params[0], 51, n_mixtures, temp=temp, sigma_temp=sigma_temp)
        performance.append(new_pose[0])
        steps += 1
    return np.array(performance)

In [ ]:
seeds = range(0,len(X_test), int(len(X_test)/10))

test_performances = [generate_performance(model,X_test[seed], steps_limit=100) for seed in seeds]

In [ ]:
longer_performance = generate_performance(model,X_test[1500], steps_limit=5000)

In [ ]:
show_animation(longer_performance)

In [ ]:
show_animation(test_performances[0])

In [ ]:
show_animation(test_performances[1])

In [ ]:
show_animation(test_performances[2])

In [ ]:
show_animation(test_performances[3])

In [ ]:
show_animation(test_performances[4])

In [ ]:
show_animation(test_performances[5])

In [ ]:
show_animation(test_performances[6])

In [ ]:
show_animation(test_performances[7])

In [ ]:
show_animation(test_performances[8])

In [ ]:
show_animation(test_performances[9])